# <center> Data Science Summer School - Split '16 </center>
(c) 2016 Martin Tutek

version: 0.1

kernel: Python 2.7

# <center>From tree to forest</center>

<center>Or how I learned to stop worrying and love overfitting.</center>

In [ ]:
# Python imports & notebook setup
# all plots will appear in the notebook
%matplotlib inline 
import warnings
warnings.filterwarnings('ignore')

import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
#import xgboost as xgb
import seaborn as sns; sns.set() # make all the plots prettier
from ipywidgets import interact, interactive, fixed # more plotting magic

from matplotlib import style
style.use('fivethirtyeight') # prettier-plots++

# sklearn models & dataset wrangling
from sklearn import tree 
from sklearn import ensemble 
from sklearn import metrics
from sklearn.cross_validation import train_test_split, StratifiedKFold, KFold
from sklearn.datasets import make_moons # toy dataset generator
from collections import Counter

# make everything reproducible
np.random.seed(12345)

# Motivation
> *Random **what**? Why not use Deep Learning™*

> _**What** boosting? I use AutoDifferentiation©, I don't need to know about gradients._

**Actually**, there still exist cases where deep learning is *not* state of the art. Main things that those cases have in common are being **regression problems**, having a **low number of target classes**, largely **uncorrelated** features and/or having a **small number of training samples**.

**Proof:**

## Taken from [eXtreme Gradient Boosting repo](https://github.com/dmlc/xgboost/blob/master/demo/README.md)
### Machine Learning Challenge Winning Solutions -- all in the last ~year

XGBoost is extensively used by machine learning practitioners to create state of art data science solutions,
this is a list of machine learning winning solutions with XGBoost.
Please send pull requests if you find ones that are missing here.

- Vlad Sandulescu, Mihai Chiru, 1st place of the [KDD Cup 2016 competition](https://kddcup2016.azurewebsites.net). Link to [the arxiv paper](http://arxiv.org/abs/1609.02728).
- Marios Michailidis, Mathias Müller and HJ van Veen, 1st place of the [Dato Truely Native? competition](https://www.kaggle.com/c/dato-native). Link to [the Kaggle interview](http://blog.kaggle.com/2015/12/03/dato-winners-interview-1st-place-mad-professors/).
- Vlad Mironov, Alexander Guschin, 1st place of the [CERN LHCb experiment Flavour of Physics competition](https://www.kaggle.com/c/flavours-of-physics). Link to [the Kaggle interview](http://blog.kaggle.com/2015/11/30/flavour-of-physics-technical-write-up-1st-place-go-polar-bears/).
- Josef Slavicek, 3rd place of the [CERN LHCb experiment Flavour of Physics competition](https://www.kaggle.com/c/flavours-of-physics). Link to [the Kaggle interview](http://blog.kaggle.com/2015/11/23/flavour-of-physics-winners-interview-3rd-place-josef-slavicek/).
- Mario Filho, Josef Feigl, Lucas, Gilberto, 1st place of the [Caterpillar Tube Pricing competition](https://www.kaggle.com/c/caterpillar-tube-pricing). Link to [the Kaggle interview](http://blog.kaggle.com/2015/09/22/caterpillar-winners-interview-1st-place-gilberto-josef-leustagos-mario/).
- Qingchen Wang, 1st place of the [Liberty Mutual Property Inspection](https://www.kaggle.com/c/liberty-mutual-group-property-inspection-prediction). Link to [the Kaggle interview](http://blog.kaggle.com/2015/09/28/liberty-mutual-property-inspection-winners-interview-qingchen-wang/).
- Chenglong Chen, 1st place of the [Crowdflower Search Results Relevance](https://www.kaggle.com/c/crowdflower-search-relevance). [Link to the winning solution](https://www.kaggle.com/c/crowdflower-search-relevance/forums/t/15186/1st-place-winner-solution-chenglong-chen/).
- Alexandre Barachant (“Cat”) and Rafał Cycoń (“Dog”), 1st place of the [Grasp-and-Lift EEG Detection](https://www.kaggle.com/c/grasp-and-lift-eeg-detection). Link to [the Kaggle interview](http://blog.kaggle.com/2015/10/12/grasp-and-lift-eeg-winners-interview-1st-place-cat-dog/).
- Halla Yang, 2nd place of the [Recruit Coupon Purchase Prediction Challenge](https://www.kaggle.com/c/coupon-purchase-prediction). Link to [the Kaggle interview](http://blog.kaggle.com/2015/10/21/recruit-coupon-purchase-winners-interview-2nd-place-halla-yang/).
- Owen Zhang, 1st place of the [Avito Context Ad Clicks competition](https://www.kaggle.com/c/avito-context-ad-clicks). Link to [the Kaggle interview](http://blog.kaggle.com/2015/08/26/avito-winners-interview-1st-place-owen-zhang/).
- Keiichi Kuroyanagi, 2nd place of the [Airbnb New User Bookings](https://www.kaggle.com/c/airbnb-recruiting-new-user-bookings). Link to [the Kaggle interview](http://blog.kaggle.com/2016/03/17/airbnb-new-user-bookings-winners-interview-2nd-place-keiichi-kuroyanagi-keiku/).
- Marios Michailidis, Mathias Müller and Ning Situ, 1st place [Homesite Quote Conversion](https://www.kaggle.com/c/homesite-quote-conversion). Link to [the Kaggle interview](http://blog.kaggle.com/2016/04/08/homesite-quote-conversion-winners-write-up-1st-place-kazanova-faron-clobber/).

### A different point of view
- Regression
- Binary classification
- Binary classification
- Binary classification
- Regression
- Ranking (can be treated as regression)
- Regression (ordinal low-dimensional classification)
- Binary classification
- Binary classification
- Binary classification
- Ranking (can be treated as regression)
- Binary classification

### So... how do they work?
> *"We are only as strong as we are united, as weak as we are divided."*

In [ ]:
# Generate a toy dataset similar to the one we used
def noisy_sine(x):
    return np.sin(x) + np.random.normal(scale=0.2, size=x.shape)

x = np.linspace(0, 2*np.pi, num = 200)
y = noisy_sine(x)

x_test = np.linspace(2*np.pi, 2.5*np.pi, num=50)
y_test = noisy_sine(x_test)

plt.plot(x, np.sin(x), linewidth=3, c='g', alpha = 0.5, label = '$sin(x)$')
plt.plot(x_test, np.sin(x_test), linewidth=3, c='g', alpha = 0.5)

plt.scatter(x, noisy_sine(x), linewidth=1, label = '$Train$')
plt.scatter(x_test, y_test, c='r', label = '$Test$')

plt.legend(loc='upper center')
plt.show()

In [ ]:
# This goes later

x = x.reshape(x.shape[0], 1)
x_test = x_test.reshape(x_test.shape[0], 1)

# Decide which models we are going to use
model_names = ["Decision Tree", "Random Forests", "Gradient Boosting"]

regression_models = {
    "Decision Tree" : tree.DecisionTreeRegressor, 
    "Gradient Boosting" : ensemble.GradientBoostingRegressor, 
    "Random Forests" : ensemble.RandomForestRegressor
}

model_params = {
    "Decision Tree" : {
        'max_depth' : 5
    },
    "Gradient Boosting" : {
        'max_depth' : 5, 'n_estimators' : 10
    },
    "Random Forests" : {
        'max_depth' : 5, 'n_estimators' : 10
    }
}

# create a 1x3 grid of plots
f, axarr = plt.subplots(3, 1, figsize=(6, 15))

for idx, regression_model in enumerate(model_names):
    # This is some dark magic you should remind me to explain if I forget
    model = regression_models[regression_model](**model_params[regression_model])
    model.fit(x, y)
    
    y_pred = model.predict(x_test)
    y_train_pred = model.predict(x)
    
    train_error = metrics.mean_squared_error(y, y_train_pred)
    test_error = metrics.mean_squared_error(y_test, y_pred)
    print regression_model, 'Train MSE =', train_error, 'Test MSE', test_error 
    
    # plot the test & train set
    axarr[idx].scatter(x, y, c='b')
    axarr[idx].scatter(x_test, y_test, c='r')
    
    axarr[idx].plot(x, y_train_pred, c="g", linewidth = 2)
    axarr[idx].plot(x_test, y_pred, c="g", label=regression_model, linewidth = 2, alpha = 1)
    axarr[idx].set_title(regression_model)

plt.show()

### OK, so it's not that simple

#### Q: What's wrong? (2 key points)
- Not enough signal
- What are the other model parameters?

#### To the documentation!

- [RandomForestRegressor](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html)

- [RandomForestClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html)

    - **n_estimators=10:** The number of trees used in the ensemble
    - **criterion='gini':** Do we **really** care? (useful to change around)
    - **max_features :** Max number of features to consider in each split - default = $\sqrt{n}$, where $n$ is the number of features (default is pretty good)
    - **max_depth :** Maximum depth of each tree in the ensemble (useful to change around)
    - **min_samples_split :** Not the minimum number of samples in Split - this is the minimum number of samples required in a node to continue splitting. Default = 2 (default is pretty good)
    - **class_weight :** Balancing the datasets - dict, list of dicts, “balanced”, “balanced_subsample” - worth considering if you have highly imbalanced classes. *(Just for classification)*

- [GradientBoostingRegressor](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingRegressor.html) 

- [GradientBoostingClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingClassifier.html) 

    - More or less the same tricks as Random Forests, except...
    - **learning_rate :** This is NOT the step we take according to the gradient of the error function, like in gradient descent. This is the contribution for each $tree \in n_{estimators}$ in the total ensemble! There is a trade-off between the learning rate and the number of estimators - as we decrease the learning rate, our generalization becomes better, however we need a larger amount of trees, which slows down the training.
    - **subsample :** The fraction of samples to be used for fitting the individual base learners. Choosing subsample < 1.0 (bagging) leads to a reduction of variance and an increase in bias.

#### Or, even better, to the [ensemble methods page](http://scikit-learn.org/stable/modules/ensemble.html):
### Bagging vs Boosting
#### Bagging methods
In bagging, we choose random subsets of the dataset (samples!), train several 'weak' (simple) learners and aggregate (average) their results. The simple learners are trained independently of each other and the process can herefore be easily parallelized. It is recommended to use subsets of features as well as samples as it leads to better generalization. 

**Q:** Can you think of any exception to the last sentence?
<img style="float: center;" src="img/bagging.png", width="70%" height="70%">

#### Boosting methods
In boosting, the weak learners are trained sequentially with a goal to reduce the error of the previous learner(s) by adding weight to previously misclassified samples. This leads to each following learner correcting the mistakes of the previous one. 

## Case study - [Kaggle bike sharing demand](https://www.kaggle.com/c/bike-sharing-demand)

### Forecast use of a city bikeshare system

#### Data Fields
- datetime - hourly date + timestamp  
- season -  1 = spring, 2 = summer, 3 = fall, 4 = winter 
- holiday - whether the day is considered a holiday
- workingday - whether the day is neither a weekend nor holiday
- weather 
  - 1: Clear, Few clouds, Partly cloudy, Partly cloudy 
  - 2: Mist + Cloudy, Mist + Broken clouds, Mist + Few clouds, Mist 
  - 3: Light Snow, Light Rain + Thunderstorm + Scattered clouds, Light Rain + Scattered clouds 
  - 4: Heavy Rain + Ice Pallets + Thunderstorm + Mist, Snow + Fog 
- temp - temperature in Celsius
- atemp - "feels like" temperature in Celsius
- humidity - relative humidity
- windspeed - wind speed
- casual - number of non-registered user rentals initiated
- registered - number of registered user rentals initiated
- count - number of total rentals

In [ ]:
# Load the data and print out the basics
train_source = 'dataset/bikes.csv'
bikes = pd.read_csv(train_source)
print bikes.shape
print bikes.head()

In [ ]:
original_features = bikes.columns.values[1:-3] 
# 'casual' and 'registered' sum up to 'count', and complicate things too much for now
target = bikes.columns.values[-1] 
print original_features, target

#### Working with dates

In [ ]:
dt = pd.DatetimeIndex(bikes['datetime']) # parse datetime into python's datetime object
bikes.set_index(dt, inplace=True)

# This allows us to add some features 
bikes['day'] = dt.day
bikes['month'] = dt.month
bikes['year'] = dt.year
bikes['hour'] = dt.hour
bikes['dow'] = dt.dayofweek
bikes['woy'] = dt.weekofyear

features = np.concatenate((original_features, bikes.columns.values[12:]))
print features

### Always plot the data

Pandas resample is relatively weakly documented - however is really useful when working with time-series or sequential data: [Stackoverflow post](http://stackoverflow.com/questions/17001389/pandas-resample-documentation)

In [ ]:
monthly_plot = bikes.resample('M').sum().plot(title="Rentals by month", y = "count", linewidth=1, legend=None)
plt.show()

In [ ]:
by_hour = bikes.groupby('hour')[['count']].agg(sum)
plt.plot(by_hour, linewidth=1)
plt.title("Rentals by hour in day")
plt.show()

In [ ]:
def get_rmsle(y_pred, y_actual):
    # Root mean square logarithmic arror - due to the sums otherwise being too large
    diff = np.log(y_pred + 1) - np.log(y_actual + 1)
    mean_error = np.square(diff).mean()
    return np.sqrt(mean_error)

def train_test_split_day(data, cutoff_day=15):
    train = data[data['day'] <= cutoff_day]
    test = data[data['day'] > cutoff_day]

    return train, test

def train_test_split_last_n(data, cutoff=1000):
    train = data[:-cutoff]
    test = data[-cutoff:]
    
    return train, test    

def kfold(model, bikes, features, target = 'count', k = 5):
    # kfold cross-validation
    skf = KFold(len(bikes), k, shuffle=True)
    
    rmslerrs = []
    
    for train, test in skf:
        bikes_train, bikes_test = bikes.iloc[train], bikes.iloc[test]
        model.fit(bikes_train[features], bikes_train[target])
        predictions = model.predict(bikes_test[features])

        RMSLE = get_rmsle(predictions, bikes_test[target])
        rmslerrs.append(RMSLE)
    
    print "The Average RMSLE = {}, stdev = {}".format(np.mean(rmslerrs), np.std(rmslerrs))
    

def dataset_performance(model, bikes_train, bikes_test, features, target = 'count', mode = 'lastn'):
    modes = ['lastn', 'day']
    
    model.fit(bikes_train[features], bikes_train[target])
    predictions = model.predict(bikes_test[features])
    
    RMSLE = get_rmsle(predictions, bikes_test[target])
    print "The Root mean squared log error is {}".format(RMSLE)
    
    # plotting 
    if mode == 'lastn':
        # Assuming datetime index, otherwise will break
        bikes_test['count_pred'] = predictions
        grouped_by_day = bikes_test.resample('D').sum() 
        indices = range(len(grouped_by_day.index))
        plt.plot(grouped_by_day.index, grouped_by_day['count'], c='g', label='Actual values', linewidth=2, alpha=0.5)
        plt.plot(grouped_by_day.index, grouped_by_day['count_pred'], c='b', label='Predicted values', linewidth=1)
        bikes_test.drop(['count_pred'], axis=1, inplace=True)
        plt.legend()
        plt.show()
    elif mode == 'day':
        bikes_test['count_pred'] = predictions
        grouped_by_day = bikes_test.resample('D').sum() 
        indices = range(len(grouped_by_day.index))
        plt.scatter(grouped_by_day.index, grouped_by_day['count'], c='g', label='Actual values', linewidth=2, alpha=0.5)
        plt.scatter(grouped_by_day.index, grouped_by_day['count_pred'], c='b', label='Predicted values', linewidth=1)
        bikes_test.drop(['count_pred'], axis=1, inplace=True)
        plt.legend(loc='upper left')
        plt.show()
    else:
        print "Skipping plotting, unknown mode: {}, supported modes for plot: {}.".format(mode, ', '.join(modes))
    
    return RMSLE
    

In [ ]:
# Feature correlation matrix
bikes[features].corr()

In [ ]:
# Do we want to keep all of the features? Do we want to add some?
print features

In [ ]:
model_names = ["Decision Tree", "Random Forests", "Gradient Boosting"]

regression_models = {
    "Decision Tree" : tree.DecisionTreeRegressor, 
    "Gradient Boosting" : ensemble.GradientBoostingRegressor, 
    "Random Forests" : ensemble.RandomForestRegressor
}

model_params = {
    "Decision Tree" : {
        'max_depth' : 5
    },
    "Gradient Boosting" : {
        'max_depth' : 5, 'n_estimators' : 10
    },
    "Random Forests" : {
        'max_depth' : 5, 'n_estimators' : 10
    }
}

modes = {
    'lastn' : train_test_split_last_n, 
    'day' : train_test_split_day
}

mode = 'lastn'

cross_validate_best = True # set to false if you don't want to run cross-validation

# Split data into train / 'test'
data_train, data_test = modes[mode](bikes) # Option: last n, or last x days in month

# Evaluate all models (Option: evaluate just one for speed)
model_scores = {}
for model_name in model_names:
    params = model_params[model_name]
    print "Running {}, params = {}".format(model_name, str(params))
    model = regression_models[model_name](**params)
    
    err = dataset_performance(model, data_train, data_test, features, mode=mode)
    model_scores[model_name] = err
    
# Cross-validate best model?
if cross_validate_best:
    sorted_scores = sorted(model_scores.items(), key=lambda t:t[1])
    best_model = sorted_scores[0][0]

    print "Running k-fold cross validation on best model: {}".format(best_model)
    m = regression_models[best_model](**model_params[best_model])
    kfold(m, bikes, features)

### So, what did the winning model use? 

### Is it all really that good?

**A:** Yes and ... no. Gradient boosted models are really good at winning online competitions - often due to overfitting the test data (yes, this is possible), however they are not as useful in real life. 

If you check the open-sourced top solutions for some of the competitions, many of them rely heavily on **feature engineering** (which is good), but sometimes also produces unintuitive features which perform well on that train/test split. Another disadvantage is that running **massive ensembles** of cascaded models and various ensembling techniques almost always proves to be better - however those models lose any resemblance of interpretability.

A really important part of running gradient boosted models is preventing overfitting - which proves to be really hard ($1e-5$ differences in learning rate can be crucial), and the top libraries for gradient boosting (XGBoost, etc) rely heavily on various regularization techniques (heuristics, early stopping) and optimizations in runtime (to allow larger models).

### Extra: Useful tricks with forests
#### Feature importance

In [ ]:
forest = ensemble.RandomForestRegressor(max_depth = 8, n_estimators = 100)
forest.fit(bikes[features], bikes[target])

importances = forest.feature_importances_
std = np.std([tree_.feature_importances_ for tree_ in forest.estimators_], axis=0)
print "Features: ", ', '.join(features)

indices = np.argsort(importances)[::-1]
feature_names = features[indices]
# Print the feature ranking
print("Feature ranking:")

for f in range(bikes[features].shape[1]):
    print("Feature #%d: %s importance: %f" % (f+1, feature_names[f], importances[indices[f]]))

# Plot the feature importances of the forest
plt.figure()
plt.title("Feature importances")
plt.bar(range(bikes[features].shape[1]), importances[indices],
       color="r", yerr=std[indices], align="center")
plt.xticks(range(bikes[features].shape[1]), range(1, bikes[features].shape[1] + 1))
plt.xlim([-1, bikes[features].shape[1]])
plt.show()

#### You can actually get the trees from the forest

In [ ]:
gbm = ensemble.GradientBoostingRegressor(learning_rate=0.01, max_depth = 8, n_estimators = 300)
gbm.fit(bikes[features], bikes[target])

top_n = 5

for idx, estimator in enumerate(gbm.estimators_[:top_n]):
    print estimator